In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
!pip install pytrials

import requests
import json
from pytrials.client import ClinicalTrials
import os
import re
from requests.exceptions import ChunkedEncodingError, ConnectionError, Timeout
import time
from tqdm import tqdm

In [ ]:
path_reactions = '/content/drive/My Drive/Data_MedEffect/reactions.txt'
path_report_drug_indication = '/content/drive/My Drive/Data_MedEffect/report_drug_indication.txt'
path_reports = '/content/drive/My Drive/Data_MedEffect/reports.txt'

In [ ]:
def strip_quotes(x):
    return x.strip('"') if isinstance(x, str) else x

In [ ]:
dtype_map = {3: str, 4: str}
df_reactions = pd.read_csv(path_reactions, delimiter = "$", header=None, dtype=dtype_map)
df_reactions.columns = ["REACTION_ID", "REPORT_ID", "DURATION", "DURATION_UNIT_ENG", "DURATION_UNIT_FR", "PT_NAME_ENG", "PT_NAME_FR", "SOC_NAME_ENG", "SOC_NAME_FR", "MEDDRA_VERSION"]
df_reactions = df_reactions.applymap(strip_quotes)
df_reactions.drop(columns=["DURATION_UNIT_FR", "PT_NAME_FR", "SOC_NAME_FR", "MEDDRA_VERSION", "REACTION_ID", "DURATION", "DURATION_UNIT_ENG"], inplace=True)
pd.set_option('display.width', 500)
print(df_reactions)

         REPORT_ID         PT_NAME_ENG                                       SOC_NAME_ENG
0                1            Pruritus             Skin and subcutaneous tissue disorders
1                1              Oedema  General disorders and administration site cond...
2                2           Urticaria             Skin and subcutaneous tissue disorders
3                3            Erythema             Skin and subcutaneous tissue disorders
4                3  Periorbital oedema                                      Eye disorders
...            ...                 ...                                                ...
4179584  906931316            Vomiting                         Gastrointestinal disorders
4179585  906931316    Weight decreased                                     Investigations
4179586  906931316    Weight increased                                     Investigations
4179587  906931316                Acne             Skin and subcutaneous tissue disorders
4179588  9

In [ ]:
df_drugs= pd.read_csv(path_report_drug_indication, delimiter = "$", header=None)
df_drugs.columns = ["REPORT_DRUG_ID", "REPORT_ID", "DRUG_PRODUCT_ID", "DRUGNAME", "INDICATION_NAME_ENG", "INDICATION_NAME_FR"]
df_drugs = df_drugs.applymap(strip_quotes)
df_drugs.drop(columns=["INDICATION_NAME_FR", "REPORT_DRUG_ID", "DRUG_PRODUCT_ID"], inplace=True)
print(df_drugs)
pd.set_option('display.max_columns', None)

         REPORT_ID                      DRUGNAME                  INDICATION_NAME_ENG
0           300198           NOVO VENLAFAXINE XR                           Depression
1           300548               BENADRYL ELIXIR  Product used for unknown indication
2           300488                      ELOXATIN                         Colon cancer
3           301050  TWINJECT 0.3MG AUTO-INJECTOR                Anaphylactic reaction
4           300841               APO-PROPAFENONE  Product used for unknown indication
...            ...                           ...                                  ...
2988921    1069622                       ENHERTU          HER2 positive breast cancer
2988922     972418                       XELJANZ  Product used for unknown indication
2988923     972418                    CETIRIZINE  Product used for unknown indication
2988924     972418                     PLAQUENIL                 Rheumatoid arthritis
2988925    1070830                       ENHERTU      

In [ ]:
dtype_map = {1: str, 5: str,16: str,39: str,40: str,41: str}
df_reports = pd.read_csv(path_reports, delimiter = "$", header=None, dtype=dtype_map)
df_reports.columns = [
    "REPORT_ID", "REPORT_NO", "VERSION_NO", "DATRECEIVED", "DATINTRECEIVED", "MAH_NO",
    "REPORT_TYPE_CODE", "REPORT_TYPE_ENG", "REPORT_TYPE_FR", "GENDER_CODE", "GENDER_ENG",
    "GENDER_FR", "AGE", "AGE_Y", "AGE_UNIT_ENG", "AGE_UNIT_FR", "OUTCOME_CODE",
    "OUTCOME_ENG", "OUTCOME_FR", "WEIGHT", "WEIGHT_UNIT_ENG", "WEIGHT_UNIT_FR", "HEIGHT",
    "HEIGHT_UNIT_ENG", "HEIGHT_UNIT_FR", "SERIOUSNESS_CODE", "SERIOUSNESS_ENG", "SERIOUSNESS_FR",  "DEATH",
    "DISABILITY", "CONGENITAL_ANOMALY", "LIFE_THREATENING", "HOSP_REQUIRED",
    "OTHER_MEDICALLY_IMP_COND", "REPORTER_TYPE_ENG", "REPORTER_TYPE_FR",
    "SOURCE_CODE", "SOURCE_ENG","SOURCE_FR", "E2B_IMP_SAFETYREPORT_ID",
    "AUTHORITY_NUMB", "COMPANY_NUMB"]
df_reports = df_reports.applymap(strip_quotes)

def convert_age(row):
    if row["AGE_UNIT_ENG"] == "Years":
        return row["AGE"]
    elif row["AGE_UNIT_ENG"] == "Months":
        return row["AGE"] / 12
    elif row["AGE_UNIT_ENG"] == "Weeks":
        return row["AGE"] / 52
    elif row["AGE_UNIT_ENG"] == "Days":
        return row["AGE"] / 365
    elif row["AGE_UNIT_ENG"] == "Hours":
        return row["AGE"] / 8760
    elif row["AGE_UNIT_ENG"] == "Decade":
        return row["AGE"]*10
    else:
        return row["AGE"]

def convert_height(row):
    if row["HEIGHT_UNIT_ENG"] == "Centimeter":
        return row["HEIGHT"] / 100
    if row["HEIGHT_UNIT_ENG"] == "Inch":
        return row["HEIGHT"] * 0.0254
    else:
        return row["HEIGHT"]

# Function to convert weight to kilograms
def convert_weight(row):
    if row["WEIGHT_UNIT_ENG"] == "Pound":
        return row["WEIGHT"] * 0.453592
    else:
        return row["WEIGHT"]

df_reports["AGE"] = df_reports.apply(lambda row: convert_age(row), axis=1)
df_reports["HEIGHT_M"] = df_reports.apply(lambda row: convert_height(row), axis=1)
df_reports["WEIGHT_KG"] = df_reports.apply(lambda row: convert_weight(row), axis=1)


df_reports.drop(columns=["VERSION_NO", "DATINTRECEIVED", "MAH_NO",
    "REPORT_TYPE_FR", "GENDER_FR", "AGE_Y", "AGE_UNIT_FR", "OUTCOME_CODE","GENDER_CODE",
    "OUTCOME_FR", "WEIGHT_UNIT_FR", "HEIGHT_UNIT_FR", "SERIOUSNESS_FR", "REPORTER_TYPE_FR",
    "SOURCE_CODE", "SOURCE_FR", "E2B_IMP_SAFETYREPORT_ID", "AUTHORITY_NUMB", "COMPANY_NUMB", "REPORTER_TYPE_ENG",
    "SOURCE_ENG", "REPORT_TYPE_CODE", "REPORT_NO", "OUTCOME_ENG", "SERIOUSNESS_CODE", "DISABILITY",
    "REPORT_TYPE_ENG", "AGE_UNIT_ENG", "HEIGHT_UNIT_ENG", "WEIGHT_UNIT_ENG", "CONGENITAL_ANOMALY", "HOSP_REQUIRED", "WEIGHT", "HEIGHT", "DATRECEIVED", "LIFE_THREATENING", "OTHER_MEDICALLY_IMP_COND", "DEATH"], inplace=True)

print(df_reports)


         REPORT_ID GENDER_ENG   AGE SERIOUSNESS_ENG  HEIGHT_M  WEIGHT_KG
0                1     Female  62.0     Not Serious      1.55       66.0
1                2     Female  70.0     Not Serious       NaN        NaN
2                3       Male  83.0         Serious      1.73       44.0
3                4     Female  78.0         Serious       NaN        NaN
4                5     Female  55.0     Not Serious       NaN        NaN
...            ...        ...   ...             ...       ...        ...
1090228  907053719       Male   NaN         Serious       NaN        NaN
1090229  907053726     Female  62.0         Serious       NaN        NaN
1090230  907053728     Female  70.0         Serious       NaN        NaN
1090231  907053730     Female   NaN         Serious       NaN        NaN
1090232  907053734     Female  46.0     Not Serious       NaN       70.0

[1090233 rows x 6 columns]


In [ ]:
acute_myocardial_infarction = ["Acute myocardial infarction","Myocardial infarction"]
heart_failure = ["Left ventricular failure", "Cardiac failure congestive", "Cardiac failure chronic", "Cardiac failure", "Right ventricular failure"]
hypertension = ["Hypertension", "Blood pressure management", "Blood pressure abnormal", "HIGH BLOOD PRESSURE MEDS", "Blood pressure increased"]
ischemic_heart_disease = ["Myocardial ischaemia", "Ischaemic heart disease prophylaxis", "Ischaemic cardiomyopathy", "Coronary artery disease", "Arteriosclerosis coronary artery"]
stroke = ["Transient ischaemic attack","Ischaemic stroke","Haemorrhagic stroke","Middle cerebral artery stroke","Embolic stroke","Lacunar stroke","Thrombotic stroke"]
chronic_obstructive_pulmonary = ["Chronic obstructive pulmonary disease", "Obstructive airways disorder", "Infective exacerbation of chronic obstructive airways", "Emphysema", "Chronic respiratory disease"]
diabetes_names = ["Type 2 diabetes mellitus", "Type 1 diabetes mellitus", "Diabetes mellitus", "Diabetes", "diabetes"]

df_acute_myocardial_infarction = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(acute_myocardial_infarction)]
df_heart_failure = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(heart_failure)]
df_hypertension = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(hypertension)]
df_ischemic_heart_disease = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(ischemic_heart_disease)]
df_stroke = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(stroke)]
df_chronic_obstructive_pulmonary = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(chronic_obstructive_pulmonary)]
df_diabetes = df_drugs[df_drugs['INDICATION_NAME_ENG'].isin(diabetes_names)]
print(df_diabetes)

df_heart_diseases = {"Myocardial infarction": df_acute_myocardial_infarction, "Heart failure":df_heart_failure, "Hypertension": df_hypertension, "Ischemic heart disease": df_ischemic_heart_disease, "Stroke": df_stroke}

         REPORT_ID                        DRUGNAME       INDICATION_NAME_ENG
309         316858                     AVANDIA 4MG         Diabetes mellitus
747         333113                LANTUS   -(VIAL)  Type 1 diabetes mellitus
784         339922                         ETHANOL  Type 2 diabetes mellitus
893         306970  NOVOLIN GE NPH INJ SUS 100U/ML  Type 1 diabetes mellitus
1101        354424                         HUMALOG         Diabetes mellitus
...            ...                             ...                       ...
2987878  907051727                         TRESIBA         Diabetes mellitus
2987884  905973473                   CANAGLIFLOZIN         Diabetes mellitus
2987926  907037065                         TRURAPI         Diabetes mellitus
2988361  906061918                          LANTUS  Type 1 diabetes mellitus
2988393  906642692                  INSULIN ASPART         Diabetes mellitus

[16867 rows x 3 columns]


In [ ]:
ctg_studies_path = '/content/drive/My Drive/Data_MedEffect/ctg-studies.csv'
ctg_studies = pd.read_csv(ctg_studies_path)
ctg_studies["Conditions"] = ctg_studies["Conditions"].str.upper()
ctg_studies["Interventions"] = ctg_studies["Interventions"].str.upper()

drug_equivalencies_path = '/content/drive/My Drive/Data_MedEffect/drug_equivalencies.csv'
drug_equivalencies = pd.read_csv(drug_equivalencies_path, skiprows=1, encoding="latin1")
drug_equivalencies.drop(columns=["RXCUI", "RXAUI", "SAB", "TTY", "CODE"], inplace =True)
drug_equivalencies["DRUGNAME"] = drug_equivalencies["DRUGNAME"].str.upper()
drug_equivalencies["STR"] = drug_equivalencies["STR"].str.upper()

# Ensure all values in are strings and handle missing values
drug_equivalencies['DRUGNAME'] = drug_equivalencies['DRUGNAME'].fillna('').astype(str)
drug_equivalencies['STR'] = drug_equivalencies['STR'].fillna('').astype(str)

# choose which disease to replace drugnames
curr_df = df_diabetes
curr_df["DRUGNAME"] = curr_df["DRUGNAME"].fillna('').astype(str)
curr_df["DRUGNAME"] = curr_df["DRUGNAME"].str.upper()
def clean_drug_name(drug):
  drug = drug.replace("(", "").replace(")", "").replace(",", "").strip()
  drug = drug.replace("-", " ").replace("&", "").replace("*", " ").strip()
  drug = drug.replace("[", "").replace("]", "").replace(",", " ").strip()
  drug = re.sub(r'\/.*?\/', '', drug)
  return drug

drug_alternates_cache = {}
def get_alternate_drug(drug):
  if drug in drug_alternates_cache:
        return drug_alternates_cache[drug]
  for _, row in drug_equivalencies.iterrows():
      if re.search(r'\b%s\b' % drug, row['DRUGNAME']) is not None:
          drug_alternates_cache[drug] = row['STR']
          return row['STR']
  drug_alternates_cache[drug] = drug
  return drug

tqdm.pandas()

save_directory = '/content/drive/My Drive/CSV/Diabetes'

print(curr_df)
curr_df['DRUGNAME'] = curr_df['DRUGNAME'].apply(clean_drug_name)

# function to apply as chunks
def process_and_save_chunk(df_chunk, chunk_index):
    # Apply the get_alternate_drug function
    df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)
    # Create the file path for saving the CSV
    csv_file_path = os.path.join(save_directory, f"diabetes_altdrug_chunk_{chunk_index}.csv")
    # Save the DataFrame chunk to a CSV file
    df_chunk.to_csv(csv_file_path, index=False)
    # Optionally print the chunk
    print(f"Chunk {chunk_index} processed and saved.")
    print(df_chunk.head())  # Print only the first few rows to avoid clutter

# Define the chunk size
chunk_size = 500
starting_chunk_index = 0
# Split the DataFrame into chunks and process each chunk
for chunk_index, start_row in enumerate(range(0, curr_df.shape[0], chunk_size), start=starting_chunk_index):
    # Get the current chunk
    df_chunk = curr_df.iloc[start_row:start_row + chunk_size]
    # Process and save the current chunk
    process_and_save_chunk(df_chunk, chunk_index)

# df_hypertension["Alternate drugname"] = df_hypertension["DRUGNAME"].progress_apply(get_alternate_drug)
# csv_file_path = os.path.join(save_directory, "hypertension_altdrug.csv")
# df_hypertension.to_csv(csv_file_path, index=False)
# for key, df in df_heart_diseases.items():

#   df['DRUGNAME'] = df['DRUGNAME'].apply(clean_drug_name)
#   df["Alternate drugname"] = df["DRUGNAME"].progress_apply(get_alternate_drug)
#   csv_file_path = os.path.join(save_directory, f"{key}_altdrug.csv")
#   df.to_csv(csv_file_path, index=False)
#   print(df)


<ipython-input-12-0df8b2cfb9c1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_df["DRUGNAME"] = curr_df["DRUGNAME"].fillna('').astype(str)
<ipython-input-12-0df8b2cfb9c1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_df["DRUGNAME"] = curr_df["DRUGNAME"].str.upper()
<ipython-input-12-0df8b2cfb9c1>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

         REPORT_ID                        DRUGNAME       INDICATION_NAME_ENG
309         316858                     AVANDIA 4MG         Diabetes mellitus
747         333113                  LANTUS    VIAL  Type 1 diabetes mellitus
784         339922                         ETHANOL  Type 2 diabetes mellitus
893         306970  NOVOLIN GE NPH INJ SUS 100U/ML  Type 1 diabetes mellitus
1101        354424                         HUMALOG         Diabetes mellitus
...            ...                             ...                       ...
2987878  907051727                         TRESIBA         Diabetes mellitus
2987884  905973473                   CANAGLIFLOZIN         Diabetes mellitus
2987926  907037065                         TRURAPI         Diabetes mellitus
2988361  906061918                          LANTUS  Type 1 diabetes mellitus
2988393  906642692                  INSULIN ASPART         Diabetes mellitus

[16867 rows x 3 columns]


100%|██████████| 500/500 [31:46<00:00,  3.81s/it]
<ipython-input-12-0df8b2cfb9c1>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 0 processed and saved.
      REPORT_ID                        DRUGNAME       INDICATION_NAME_ENG              Alternate drugname
309      316858                     AVANDIA 4MG         Diabetes mellitus                     AVANDIA 4MG
747      333113                  LANTUS    VIAL  Type 1 diabetes mellitus                  LANTUS    VIAL
784      339922                         ETHANOL  Type 2 diabetes mellitus                         ETHANOL
893      306970  NOVOLIN GE NPH INJ SUS 100U/ML  Type 1 diabetes mellitus  NOVOLIN GE NPH INJ SUS 100U/ML
1101     354424                         HUMALOG         Diabetes mellitus                  INSULIN LISPRO


100%|██████████| 500/500 [16:11<00:00,  1.94s/it]
<ipython-input-12-0df8b2cfb9c1>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


...

100%|██████████| 500/500 [02:29<00:00,  3.34it/s]
<ipython-input-12-0df8b2cfb9c1>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 32 processed and saved.
         REPORT_ID     DRUGNAME       INDICATION_NAME_ENG           Alternate drugname
2828178     919930    METFORMIN         Diabetes mellitus                    METFORMIN
2828192     939149    JARDIANCE  Type 2 diabetes mellitus    DAPAGLIFLOZIN / METFORMIN
2828239  903801416  SAXAGLIPTIN  Type 2 diabetes mellitus  DAPAGLIFLOZIN / SAXAGLIPTIN
2828255     938462    JARDIANCE         Diabetes mellitus    DAPAGLIFLOZIN / METFORMIN
2828676     943239      OZEMPIC  Type 2 diabetes mellitus                  SEMAGLUTIDE


100%|██████████| 367/367 [02:26<00:00,  2.51it/s]

Chunk 33 processed and saved.
         REPORT_ID         DRUGNAME       INDICATION_NAME_ENG     Alternate drugname
2916530     736739        NOVORAPID         Diabetes mellitus  INSULIN ASPART, HUMAN
2916611  903041202        METFORMIN  Type 2 diabetes mellitus              METFORMIN
2916934     908171  LEVEMIR PENFILL  Type 2 diabetes mellitus        INSULIN DETEMIR
2917008  903080950          HUMALOG         Diabetes mellitus         INSULIN LISPRO
2917017     908829          FORXIGA  Type 2 diabetes mellitus          DAPAGLIFLOZIN



<ipython-input-12-0df8b2cfb9c1>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


In [ ]:
# concatenate all chunks together into a single csv file
save_directory = '/content/drive/My Drive/CSV/Parkinsonism'
output_file = os.path.join(save_directory, 'parkinsonism_altdrug.csv')

# List to store DataFrames
df_list = []

# Iterate over all files in the save directory
for filename in sorted(os.listdir(save_directory)):
    if filename.startswith('parkinsonism_altdrug_chunk_') and filename.endswith('.csv'):
        file_path = os.path.join(save_directory, filename)
        # Read the CSV file into a DataFrame
        df_chunk = pd.read_csv(file_path)
        # Append the DataFrame to the list
        df_list.append(df_chunk)
        print(f"Processed {filename}")

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a single CSV file
combined_df.to_csv(output_file, index=False)

Processed parkinsonism_altdrug_chunk_0.csv
Processed parkinsonism_altdrug_chunk_1.csv
Processed parkinsonism_altdrug_chunk_2.csv
Processed parkinsonism_altdrug_chunk_3.csv
Processed parkinsonism_altdrug_chunk_4.csv
